In [1]:
import pandas as pd
import os
import re

In [ ]:
# Ruta del archivo Excel
file_path = 'data/original/Ubicación actual de las Versiones Restauradas.xlsx'

# Especifica el nombre o índice de la hoja que deseas importar
sheet_name = 'VERSIONES RESTAURADAS'  # También puedes usar el índice de la hoja, por ejemplo: 0

# Cargar la hoja especificada en un DataFrame
df_verificacion = pd.read_excel(file_path, sheet_name=sheet_name)

# Mostrar las primeras filas del DataFrame
print(df_verificacion.head())


In [ ]:
df_codigos = pd.DataFrame()

df_codigos[['Codigo', 'Resto']] = df_verificacion['Código versión restaurada'].str.extract(r'(^.*\d{6})\s*(.*)')

# Mostrar el nuevo DataFrame con las dos columnas
print(df_codigos[['Codigo', 'Resto']])

In [ ]:
# Ruta de la carpeta que contiene los archivos CSV
folder_path = 'data/original/nas_csv'

# Crear un diccionario para almacenar los DataFrames
dfs = {}

# Función para convertir el nombre del archivo a un formato válido
def valid_name(name):
    # Reemplazar caracteres no válidos y espacios por guiones bajos
    name = re.sub(r'[^a-zA-Z0-9_]', '_', name)  # Reemplazar caracteres no alfanuméricos por '_'
    name = name.replace(' ', '_')  # Reemplazar espacios por '_'
    return name

# Iterar sobre cada archivo en la carpeta
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        # Obtener la ruta completa del archivo
        file_path = os.path.join(folder_path, filename)
        
        # Leer el archivo CSV en un DataFrame
        df = pd.read_csv(file_path)
        
        # Usar el nombre del archivo (sin la extensión) como clave en el diccionario
        df_name = valid_name(os.path.splitext(filename)[0])  # Convertir a un nombre válido
        dfs[df_name] = df

# Ahora dfs contiene todos los DataFrames
# Ejemplo de cómo acceder a un DataFrame específico
# df_especifico = dfs.get('nombre_del_archivo_sin_extension')

# Mostrar los nombres de los DataFrames creados
print(dfs.keys())

In [22]:
import os
import pandas as pd

# Función para limpiar el código
def clean_code(code):
    return str(code).upper().strip()  # Convierte a mayúsculas y elimina espacios

# Función para preprocesar los archivos CSV
def preprocess_csv_files(folder_path):
    preprocessed_data = []
    # Iterar sobre los archivos en la carpeta
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            # Cargar solo la primera columna (asumiendo que es la columna con los códigos)
            df = pd.read_csv(file_path, usecols=[0], header=None)  # Cargar solo la primera columna
            df['cleaned_code'] = df[0].apply(clean_code)  # Limpiar los códigos
            preprocessed_data.append((filename, df[['cleaned_code']]))  # Guardar nombre de archivo y DataFrame limpio
    return preprocessed_data

# Función para crear un diccionario con los códigos
def create_code_map(preprocessed_data):
    code_map = {}
    for filename, df in preprocessed_data:
        for idx, row in df.iterrows():
            code = row['cleaned_code']
            if code not in code_map:
                code_map[code] = []  # Inicializa una lista para múltiples ocurrencias
            code_map[code].append((filename, idx))  # Guarda el archivo y la fila
    return code_map

# Función para verificar la existencia de un código
def verificar_existencia(codigo):
    cleaned_codigo = clean_code(codigo)  # Limpiar el código a buscar
    if cleaned_codigo in code_map:  # Verificar si está en el diccionario
        return "si", code_map[cleaned_codigo][0][0], code_map[cleaned_codigo][0][1]  # Retorna "si", el archivo y la fila
    return "no", None, None  # Retorna "no" si no se encuentra

# Ejecutar el preprocesamiento y la creación del diccionario
folder_path = 'data/original/nas_csv'  # Ruta donde están los archivos CSV
preprocessed_data = preprocess_csv_files(folder_path)  # Preprocesar los archivos CSV
code_map = create_code_map(preprocessed_data)  # Crear el diccionario de códigos

# Supongamos que df_codigos es tu DataFrame con los códigos a verificar
df_codigos = pd.DataFrame({'Codigo': ['VR F35MM 851242', 'UMT 205827 CLIP 1', 'DV 270979']})

# Aplicar la función para verificar existencia
df_codigos[['existe', 'archivo', 'fila']] = df_codigos['Codigo'].apply(verificar_existencia).apply(pd.Series)

# Mostrar el DataFrame actualizado
print(df_codigos)


              Codigo existe    archivo  fila
0    VR F35MM 851242     no       None   NaN
1  UMT 205827 CLIP 1     si  nas_2.csv   0.0
2          DV 270979     si  nas_2.csv   2.0
